<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
عیب یابی</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را 
<code>import</code>
 کنید 
 و
  فایل داده‌ها را که در پوشه‌ی 
  <code>Data</code>
ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [1]:
# Import کردن کتابخانه‌های مورد نیاز
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# خواندن فایل‌های آموزشی
print('خواندن فایل‌های داده...')
temp_train = pd.read_csv('Train/Temperature_C_train.csv')
pressure_train = pd.read_csv('Train/Pressure_kPa_train.csv')
vibaccel_train = pd.read_csv('Train/VibAccel_m_s2_train.csv')
vibvel_train = pd.read_csv('Train/VibVelocity_mm_s_train.csv')

# خواندن فایل‌های تست
temp_test = pd.read_csv('Test/Temperature_C_test.csv')
pressure_test = pd.read_csv('Test/Pressure_kPa_test.csv')
vibaccel_test = pd.read_csv('Test/VibAccel_m_s2_test.csv')
vibvel_test = pd.read_csv('Test/VibVelocity_mm_s_test.csv')

print('فایل‌ها با موفقیت خوانده شدند!')
print(f'Temperature Train: {temp_train.shape}')
print(f'Pressure Train: {pressure_train.shape}')
print(f'VibAccel Train: {vibaccel_train.shape}')
print(f'VibVelocity Train: {vibvel_train.shape}')

خواندن فایل‌های داده...
فایل‌ها با موفقیت خوانده شدند!
Temperature Train: (107100, 2)
Pressure Train: (53550, 2)
VibAccel Train: (10710, 3)
VibVelocity Train: (21420, 2)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [2]:
# تبدیل timestamp به datetime
for df in [temp_train, pressure_train, vibaccel_train, vibvel_train, temp_test, pressure_test, vibaccel_test, vibvel_test]:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)

# تعیین بازه زمانی مشترک
start_time = max([temp_train['timestamp'].min(), pressure_train['timestamp'].min(), 
                  vibaccel_train['timestamp'].min(), vibvel_train['timestamp'].min()])
end_time = min([temp_train['timestamp'].max(), pressure_train['timestamp'].max(), 
                vibaccel_train['timestamp'].max(), vibvel_train['timestamp'].max()])

print(f'بازه زمانی مشترک: {start_time} تا {end_time}')

# ایجاد timestamp مرجع (هر 1 ثانیه) بر اساس Temperature که بالاترین نرخ نمونه‌برداری را دارد
reference_timestamps = temp_train[(temp_train['timestamp'] >= start_time) & 
                                  (temp_train['timestamp'] <= end_time)]['timestamp'].values

print(f'تعداد timestamp های مرجع: {len(reference_timestamps)}')

# تابع برای interpolation و merge کردن داده‌ها
def interpolate_and_merge(sensor_df, sensor_name, reference_ts):
    # فیلتر کردن داده‌ها در بازه زمانی مشترک
    sensor_df_filtered = sensor_df[(sensor_df['timestamp'] >= start_time) & 
                                   (sensor_df['timestamp'] <= end_time)].copy()
    
    # ایجاد DataFrame مرجع
    ref_df = pd.DataFrame({'timestamp': reference_ts})
    
    # merge با استفاده از asof برای نزدیک‌ترین زمان
    merged = pd.merge_asof(ref_df.sort_values('timestamp'), 
                          sensor_df_filtered.sort_values('timestamp'), 
                          on='timestamp', direction='nearest')
    
    return merged

# ادغام داده‌های آموزشی
print('ادغام داده‌های آموزشی...')
train_merged = pd.DataFrame({'timestamp': reference_timestamps})

# ادغام Temperature
temp_merged = interpolate_and_merge(temp_train, 'Temperature_C', reference_timestamps)
train_merged['Temperature_C'] = temp_merged['Temperature_C']

# ادغام Pressure
pressure_merged = interpolate_and_merge(pressure_train, 'Pressure_kPa', reference_timestamps)
train_merged['Pressure_kPa'] = pressure_merged['Pressure_kPa']

# ادغام VibAccel (شامل target)
vibaccel_merged = interpolate_and_merge(vibaccel_train, 'VibAccel_m_s2', reference_timestamps)
train_merged['VibAccel_m_s2'] = vibaccel_merged['VibAccel_m_s2']
train_merged['faulted'] = vibaccel_merged['faulted']

# ادغام VibVelocity
vibvel_merged = interpolate_and_merge(vibvel_train, 'VibVelocity_mm_s', reference_timestamps)
train_merged['VibVelocity_mm_s'] = vibvel_merged['VibVelocity_mm_s']

print(f'داده ادغام شده آموزشی: {train_merged.shape}')
print(f'مقادیر null در هر ستون:')
print(train_merged.isnull().sum())

# نمایش توزیع کلاس‌ها
print('\nتوزیع کلاس‌ها:')
print(train_merged['faulted'].value_counts())

بازه زمانی مشترک: 2025-01-01 00:00:09 تا 2025-01-02 05:44:59
تعداد timestamp های مرجع: 107091
ادغام داده‌های آموزشی...
داده ادغام شده آموزشی: (107091, 6)
مقادیر null در هر ستون:
timestamp              0
Temperature_C       2122
Pressure_kPa        4180
VibAccel_m_s2       5780
faulted                0
VibVelocity_mm_s    4835
dtype: int64

توزیع کلاس‌ها:
faulted
normal     59775
faulted    47316
Name: count, dtype: int64


In [3]:
# پیش‌پردازش و مهندسی ویژگی
print('شروع پیش‌پردازش...')

# حذف ردیف‌هایی که هیچ مقدار سنسوری ندارند
train_cleaned = train_merged.dropna(subset=['faulted']).copy()

# پُر کردن مقادیر گمشده با میانگین
feature_columns = ['Temperature_C', 'Pressure_kPa', 'VibAccel_m_s2', 'VibVelocity_mm_s']

imputer = SimpleImputer(strategy='mean')
train_cleaned[feature_columns] = imputer.fit_transform(train_cleaned[feature_columns])

# ایجاد ویژگی‌های جدید
print('ایجاد ویژگی‌های جدید...')

# ویژگی‌های آماری روی پنجره‌های زمانی
window_sizes = [5, 10, 20]  # پنجره‌های 5، 10 و 20 ثانیه

for window in window_sizes:
    for col in feature_columns:
        # میانگین متحرک
        train_cleaned[f'{col}_rolling_mean_{window}'] = train_cleaned[col].rolling(window=window, min_periods=1).mean()
        # انحراف معیار متحرک
        train_cleaned[f'{col}_rolling_std_{window}'] = train_cleaned[col].rolling(window=window, min_periods=1).std()
        # حداکثر متحرک
        train_cleaned[f'{col}_rolling_max_{window}'] = train_cleaned[col].rolling(window=window, min_periods=1).max()
        # حداقل متحرک
        train_cleaned[f'{col}_rolling_min_{window}'] = train_cleaned[col].rolling(window=window, min_periods=1).min()

# ویژگی‌های ترکیبی
train_cleaned['temp_pressure_ratio'] = train_cleaned['Temperature_C'] / (train_cleaned['Pressure_kPa'] + 1e-6)
train_cleaned['vib_total'] = train_cleaned['VibAccel_m_s2'] + train_cleaned['VibVelocity_mm_s']
train_cleaned['vib_ratio'] = train_cleaned['VibAccel_m_s2'] / (train_cleaned['VibVelocity_mm_s'] + 1e-6)

# حذف ستون timestamp
X_train = train_cleaned.drop(['timestamp', 'faulted'], axis=1)
y_train = train_cleaned['faulted']

# تبدیل target به عددی
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

print(f'شکل داده نهایی: {X_train.shape}')
print(f'تعداد ویژگی‌ها: {X_train.shape[1]}')
print(f'کلاس‌ها: {le.classes_}')

# جایگزینی مقادیر inf و nan
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(X_train.mean())

print('پیش‌پردازش تکمیل شد!')

شروع پیش‌پردازش...
ایجاد ویژگی‌های جدید...
شکل داده نهایی: (107091, 55)
تعداد ویژگی‌ها: 55
کلاس‌ها: ['faulted' 'normal']
پیش‌پردازش تکمیل شد!


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [ ]:
# آموزش مدل‌های مختلف
print('شروع آموزش مدل‌ها...')

# تقسیم داده برای اعتبارسنجی
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded)

# استانداردسازی داده‌ها
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val_split)

# مدل‌های مختلف
models = {
    'RandomForest': RandomForestClassifier(n_estimators=200, max_depth=15, 
                                          min_samples_split=5, min_samples_leaf=2,
                                          random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=150, max_depth=8,
                                                   learning_rate=0.1, random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000)
}

# آموزش و ارزیابی مدل‌ها
best_model = None
best_score = 0
model_results = {}

for name, model in models.items():
    print(f'\nآموزش {name}...')
    
    # آموزش مدل
    if name == 'LogisticRegression':
        model.fit(X_train_scaled, y_train_split)
        y_pred = model.predict(X_val_scaled)
    else:
        model.fit(X_train_split, y_train_split)
        y_pred = model.predict(X_val_split)
    
    # محاسبه Macro F1
    f1_macro = f1_score(y_val_split, y_pred, average='macro')
    model_results[name] = f1_macro
    
    print(f'{name} Macro F1 Score: {f1_macro:.4f}')
    
    # نگه‌داری بهترین مدل
    if f1_macro > best_score:
        best_score = f1_macro
        best_model = model
        best_model_name = name
        if name == 'LogisticRegression':
            best_scaler = scaler

print(f'\nبهترین مدل: {best_model_name} با امتیاز: {best_score:.4f}')

# آموزش مجدد بهترین مدل روی کل داده
print('\nآموزش مجدد بهترین مدل روی کل داده‌های آموزشی...')
if best_model_name == 'LogisticRegression':
    X_train_full_scaled = scaler.fit_transform(X_train)
    best_model.fit(X_train_full_scaled, y_train_encoded)
else:
    best_model.fit(X_train, y_train_encoded)

print('آموزش مدل تکمیل شد!')

شروع آموزش مدل‌ها...

آموزش RandomForest...
RandomForest Macro F1 Score: 0.9812

آموزش GradientBoosting...
GradientBoosting Macro F1 Score: 0.9883

آموزش LogisticRegression...
LogisticRegression Macro F1 Score: 0.6625

بهترین مدل: GradientBoosting با امتیاز: 0.9883

آموزش مجدد بهترین مدل روی کل داده‌های آموزشی...


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [ ]:
from sklearn.metrics import f1_score, classification_report

# ارزیابی نهایی مدل بر روی validation set
print('ارزیابی نهایی مدل:')

# پیش‌بینی روی validation set
if best_model_name == 'LogisticRegression':
    final_pred = best_model.predict(X_val_scaled)
else:
    final_pred = best_model.predict(X_val_split)

# محاسبه امتیازهای مختلف
macro_f1 = f1_score(y_val_split, final_pred, average='macro')
normal_f1 = f1_score(y_val_split, final_pred, pos_label=0)
faulted_f1 = f1_score(y_val_split, final_pred, pos_label=1)

print(f'Macro F1 Score: {macro_f1:.4f}')
print(f'Normal F1 Score: {normal_f1:.4f}')
print(f'Faulted F1 Score: {faulted_f1:.4f}')

# نمایش گزارش کامل
print('\nClassification Report:')
target_names = ['normal', 'faulted']
print(classification_report(y_val_split, final_pred, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(y_val_split, final_pred)
print('\nConfusion Matrix:')
print(cm)

# بررسی اینکه آیا امتیاز بالای حد آستانه است
if macro_f1 >= 0.5:
    print(f'✅ مدل شرط حداقل امتیاز (0.5) را برآورده می‌کند: {macro_f1:.4f}')
else:
    print(f'❌ مدل شرط حداقل امتیاز (0.5) را برآورده نمی‌کند: {macro_f1:.4f}')

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [ ]:
# پیش‌بینی برای داده‌های تست
print('شروع پیش‌بینی روی داده‌های تست...')

# تعیین بازه زمانی مشترک برای تست
start_time_test = max([temp_test['timestamp'].min(), pressure_test['timestamp'].min(), 
                      vibaccel_test['timestamp'].min(), vibvel_test['timestamp'].min()])
end_time_test = min([temp_test['timestamp'].max(), pressure_test['timestamp'].max(), 
                    vibaccel_test['timestamp'].max(), vibvel_test['timestamp'].max()])

# ایجاد timestamp مرجع برای تست
reference_timestamps_test = temp_test[(temp_test['timestamp'] >= start_time_test) & 
                                     (temp_test['timestamp'] <= end_time_test)]['timestamp'].values

print(f'تعداد timestamp های تست: {len(reference_timestamps_test)}')

# ادغام داده‌های تست
print('ادغام داده‌های تست...')
test_merged = pd.DataFrame({'timestamp': reference_timestamps_test})

# ادغام هر سنسور
temp_test_merged = interpolate_and_merge(temp_test, 'Temperature_C', reference_timestamps_test)
test_merged['Temperature_C'] = temp_test_merged['Temperature_C']

pressure_test_merged = interpolate_and_merge(pressure_test, 'Pressure_kPa', reference_timestamps_test)
test_merged['Pressure_kPa'] = pressure_test_merged['Pressure_kPa']

vibaccel_test_merged = interpolate_and_merge(vibaccel_test, 'VibAccel_m_s2', reference_timestamps_test)
test_merged['VibAccel_m_s2'] = vibaccel_test_merged['VibAccel_m_s2']

vibvel_test_merged = interpolate_and_merge(vibvel_test, 'VibVelocity_mm_s', reference_timestamps_test)
test_merged['VibVelocity_mm_s'] = vibvel_test_merged['VibVelocity_mm_s']

# پیش‌پردازش داده‌های تست (همان فرآیند train)
print('پیش‌پردازش داده‌های تست...')

# پُر کردن مقادیر گمشده
test_merged[feature_columns] = imputer.transform(test_merged[feature_columns])

# ایجاد ویژگی‌های جدید
for window in window_sizes:
    for col in feature_columns:
        test_merged[f'{col}_rolling_mean_{window}'] = test_merged[col].rolling(window=window, min_periods=1).mean()
        test_merged[f'{col}_rolling_std_{window}'] = test_merged[col].rolling(window=window, min_periods=1).std()
        test_merged[f'{col}_rolling_max_{window}'] = test_merged[col].rolling(window=window, min_periods=1).max()
        test_merged[f'{col}_rolling_min_{window}'] = test_merged[col].rolling(window=window, min_periods=1).min()

# ویژگی‌های ترکیبی
test_merged['temp_pressure_ratio'] = test_merged['Temperature_C'] / (test_merged['Pressure_kPa'] + 1e-6)
test_merged['vib_total'] = test_merged['VibAccel_m_s2'] + test_merged['VibVelocity_mm_s']
test_merged['vib_ratio'] = test_merged['VibAccel_m_s2'] / (test_merged['VibVelocity_mm_s'] + 1e-6)

# آماده‌سازی برای پیش‌بینی
X_test = test_merged.drop(['timestamp'], axis=1)

# جایگزینی مقادیر inf و nan
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(X_test.mean())

# پیش‌بینی
if best_model_name == 'LogisticRegression':
    X_test_scaled = best_scaler.transform(X_test)
    test_predictions = best_model.predict(X_test_scaled)
else:
    test_predictions = best_model.predict(X_test)

# تبدیل پیش‌بینی‌ها به رشته
test_predictions_labels = le.inverse_transform(test_predictions)

# ایجاد DataFrame نهایی
submission = pd.DataFrame({
    'prediction': test_predictions_labels
})

print(f'تعداد پیش‌بینی‌ها: {len(submission)}')
print('توزیع پیش‌بینی‌ها:')
print(submission['prediction'].value_counts())

print('پیش‌بینی تکمیل شد!')
print('حالا میتونی سلول آخر رو اجرا کنی تا فایل result.zip ساخته بشه.')

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [ ]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)